# Imports

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from __future__ import print_function

import glob
import math
import os
import copy

import numpy as np
import pandas as pd


from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

# Classes

In [ ]:
class NN:
    def __init__(self, input_shape, units, drop_prob=0):
        """
        units: (10, 11, 12)
            => 3 couches avec 10 neurones pour la premiere
        """
        nb_output = input_shape
        self.weights = []
        for nb_neurones in units:
            self.weights.append(np.random.rand(nb_output, nb_neurones))
            self.weights.append(np.zeros(nb_neurones))
            nb_output = nb_neurones
        
        # TODO: a finir
        self.drop_prob = drop_prob
    
    def print_weights(self):
        for l in self.weights:
            print(l)

    def preactiv(self, x):
        res = x
        for i in range(0, len(self.weights), 2):
            res = self.activ(res @ self.weights[i] + self.weights[i + 1])

        return res

    def activ(self, res):
        """
        :param numpy_array res: output d'une couche
        :return l'activation LeakyReLu du resultat
        """
        res[res < 0] *= 0.001
        return res

    def loss(self, res, y):
        if len(y.shape) == 1:
            return res - np.expand_dims(y, 1)
        return res - y

    def d_activ(self, res):
        """
        :param numpy_array res: output d'une couche
        :return la derive de l'activation LeakyReLu du resultat
        """
        dres = res
        dres[dres < 0] = 0.001
        dres[dres >= 0] = 1
        return dres
    
    def forward(self, x, res_arr=None):
        """
        :param numpy_array x: input
        :param numpy_array res_arr: array ou on stock les resultats des couches intermediaires
        :return le resultat du forward et les resultats intermediaires
        """
        if res_arr:
            res_arr[0] = x
        res = x
        for h in range(0, len(self.weights), 2):  # calculate the result for the current features
            res = self.activ(res @ self.weights[h] + self.weights[h + 1])
            if res_arr:
                res_arr[h//2+1] = res
        
        if res_arr:
            return res, res_arr
        else:
            return res
    
    def backward(self, res, y, res_arr, batch_gradient, learning_rate, batch_size):
        """
        :param numpy_array res: resultat du forward
        :param numpy_array y: vraie resultat
        :param numpy_array res_arr: array ou on stock les resultats des couches intermediaires
        :param numpy_array batch_gradient: array contenant la somme des gradients calcules sur le batch
        :param integer float: learning rate
        :param integer batch_size: taille du batch
        """
        
        # calcul de l'erreur finale
        # TODO: adapter pour que ca fonctionne avec plusieurs outputs
        loss = res - np.expand_dims(y, 1)

        # update of the last neurone
        delta = loss * self.d_activ(res)
        batch_gradient[-2] += res_arr[-2].T @ delta
        # TODO: faire fonctionner le bias
        batch_gradient[-1] += np.sum(delta)

        for b in reversed(range(0, len(self.weights) - 2, 2)):  # backpropagations
            l_output = res_arr[(b // 2) + 1]  # layer output
            loss = delta @ self.weights[b + 2].T
            delta = loss * self.d_activ(l_output)
            batch_gradient[b] += res_arr[b//2].T @ delta
            # TODO: faire fonctionner le bias
            batch_gradient[b+1] += np.sum(delta)

        for k in range(len(batch_gradient)):
            self.weights[k] -= learning_rate * (1 / batch_size) * batch_gradient[k]
    
    def fit(self, x, y, step, epochs, batch_size, learning_rate, validation_datas):
        """
        :param numpy_array x: input
        :param numpy_array y: vraie resultat
        :param numpy_array res_arr: array ou on stock les resultats des couches intermediaires
        :param numpy_array batch_gradient: array contenant la somme des gradients calcules sur le batch
        :param integer float: learning rate
        :param integer batch_size: taille du batch
        """
        x_val = validation_datas[0]
        y_val = validation_datas[1]
        x = x
        y = y
        i = 0
        epo = 0
        steps_per_epoch = step // epochs

        default_batch_gradient = []
        for k in range(0, len(self.weights), 1):
            default_batch_gradient.append(np.zeros(self.weights[k].shape))
        
        default_res_arr = [np.zeros((self.weights[0].shape[0], batch_size))]
        for k in range(0, len(self.weights), 2):
            default_res_arr.append(np.zeros((self.weights[k].shape[1], batch_size)))

        for etape in tqdm(range(step)):
            i += 1
            
            # recuperation des donnees du batch
            bx = x.iloc[batch_size * i : batch_size * (i + 1)].to_numpy()
            by = y.iloc[batch_size * i : batch_size * (i + 1)].to_numpy()
            
            # store the gradient calculated osn the whole batch
            batch_gradient = copy.deepcopy(default_batch_gradient)
            # store the result after each layer
            res_arr = copy.deepcopy(default_res_arr)
            
            res, res_arr = self.forward(bx, res_arr)
            
            self.backward(res, by, res_arr, batch_gradient, learning_rate, batch_size)

            if x.shape[0] < batch_size * (i + 1) + 1:
                p = np.random.permutation(len(x))
                x = x.iloc[p]
                y = y.iloc[p]
                i = 0
                
            if etape != 0 and etape % (steps_per_epoch) == 0:
                predic_val = self.forward(x_val)[0]
                predic_train = self.forward(x.iloc[:step // epochs])
                print("Resume des erreurs sur la validation")
                print(self.loss(predic_val - y_val).describe())
        
                val_cost = ((self.loss(predic_val, y_val)) ** 2).mean()
                train_cost = ((self.loss(predic_train, y.iloc[:step // epochs])) ** 2).mean()
                
                val_rmse = math.sqrt(val_cost)
                train_rmse = math.sqrt(train_cost)
                
                print(
                    "Epoch: ",
                    epo,
                    "| loss val: ",
                    val_rmse,
                    " | loss train ",
                    train_rmse,
                )
                print("mean predic: ", predic_val.mean(), " / ", y_val.mean())
                
                epo += 1

# Recuperation des donnees

In [ ]:
# dataset = pd.read_csv("./winequality-red.csv")

# train = dataset
# validation = dataset.tail(199)

# x_train = train.drop('quality', 1)
# y_train = train.quality

# train_sata = pd.read_csv("./train_satander.csv")
# x_train = train_sata.drop("target", 1)
# x_train = x_train.drop("ID_code", 1)
# y_train = train_sata.target

cali_dataframe = pd.read_csv("./california_housing_train.csv")
x = cali_dataframe.drop("median_house_value", 1)
y = cali_dataframe.median_house_value / 100000

# wine_dataframe = pd.read_csv("./winequality-red.csv")
# x_train = wine_dataframe.drop("quality", 1)
# y_train = wine_dataframe.quality

# Normalisation et transformation

In [ ]:
def normalize(x):
    return (x - min(x)) / (max(x) - min(x))


for k in x:
    x[k] = normalize(x[k])

x_train = x.head(16000)
y_train = y.head(16000)

x_val = x.tail(1000)
y_val = y.tail(1000)
print(x_val.describe())

print("y summary")
print(y_train.describe())

# Training

In [ ]:
model = NN(x_train.shape[1], (50, 50, 50, 1), drop_prob=0.1)

In [ ]:
print("first predic: ", math.sqrt( ((model.loss(model.forward(x_val), y_val))**2).mean()))
print("Training")
model.fit(
    x_train,
    y_train,
    step=100000,
    epochs=10,
    batch_size=50,
    learning_rate=0.000000001,
    validation_datas=(x_val, y_val),
)